**<h2><font color='red'>This is a LSTM starter </h2>**
**<h2>I will definately add Cross Validation strategy in future which will make the LB score better and I will keep updating the hyper-parameters!<br>Thank You</h2>**
**<h2><font color='red'>PLEASE DON'T FORGET TO UPVOTE</h2>**

In [ ]:
import os
import glob
import shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
seed=42
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test  = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
sub   = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv') 

In [ ]:
#Train Data
uin, uout, R, C, pressure = [],[],[],[],[]

for ix in tqdm(train['breath_id'].unique()):
    rdf = train[train['breath_id']==ix]
    uin.append(np.array(rdf['u_in']).reshape(-1, 80))
    uout.append(np.array(rdf['u_out']).reshape(-1, 80))
    R.append(np.array(rdf['R']).reshape(-1, 80))
    C.append(np.array(rdf['C']).reshape(-1, 80))
    pressure.append(np.array(rdf['pressure']).reshape(-1, 80))
    
uin=np.array(uin).reshape(-1, 80)
uout=np.array(uout).reshape(-1, 80)
R=np.array(R).reshape(-1, 80)
C=np.array(C).reshape(-1, 80)
pressure=np.array(pressure).reshape(-1, 80)

X = np.stack([uin, uout, R, C], axis=-1)
Y = pressure 
print(X.shape, Y.shape)

In [ ]:
#Test Data
t_uin, t_uout, t_R, t_C = [],[],[],[]

for ix in tqdm(test['breath_id'].unique()):
    rdf = test[test['breath_id']==ix]
    t_uin.append(np.array(rdf['u_in']).reshape(-1, 80))
    t_uout.append(np.array(rdf['u_out']).reshape(-1, 80))
    t_R.append(np.array(rdf['R']).reshape(-1, 80))
    t_C.append(np.array(rdf['C']).reshape(-1, 80))
    
t_uin=np.array(t_uin).reshape(-1, 80)
t_uout=np.array(t_uout).reshape(-1, 80)
t_R=np.array(t_R).reshape(-1, 80)
t_C=np.array(t_C).reshape(-1, 80)

t_X = np.stack([t_uin, t_uout, t_R, t_C], axis=-1)
print(t_X.shape)

In [ ]:
from tensorflow.keras.layers import Dense, LSTM, Input, Activation
def create_model():
    ipt = Input(shape=(80, 4))
    x = LSTM(150, return_sequences=True)(ipt)
    x = LSTM(100, return_sequences=True)(x)
    x = LSTM(80, return_sequences=True)(x)
    x = Dense(80, activation='relu')(x)
    x = Dense(40, activation='relu')(x)
    x = Dense(1)(x)
    #x = LSTM(150, return_sequences=True)(ipt)
    #x = LSTM(100, return_sequences=True)(x)
    #x = LSTM(80, return_sequences=False)(x)
    #x = Dense(80, activation='relu')(x)
    
    return tf.keras.models.Model(ipt, x)

In [ ]:
#5-FOLD CV
from sklearn.model_selection import KFold

k=5
preds = np.zeros((sub.shape[0], 1), dtype=np.float64)
kfold = KFold(n_splits=k)
for i, (train_index, val_index) in enumerate(kfold.split(np.arange(X.shape[0]))):
    print(f"STARTING FOLD -> {i+1}")
    x_train = X[train_index]
    y_train = Y[train_index]
    x_val = X[val_index]
    y_val = Y[val_index]
    
    model = create_model()
    mc_cb = tf.keras.callbacks.ModelCheckpoint(f'./fold_{i}_weights.h5', save_weights_only=True, save_best_only=True, 
                                           monitor='val_loss', mode='min')
    model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(lr=0.0005))
    history = model.fit(x_train, y_train, epochs=150, batch_size=64, 
                        validation_data = (x_val, y_val), callbacks=[mc_cb], verbose=0)
    pd.DataFrame(history.history).plot()
    plt.grid(True)
    plt.show()
    model.load_weights(f'./fold_{i}_weights.h5')
    print("Validation Loss: ", model.evaluate(x_val, y_val))
    #prediction
    preds += model.predict(t_X).reshape(-1, 1) 
preds/=k

In [ ]:
#submission
sub['pressure'] = preds
sub.to_csv('submission.csv', index=False)
sub.head(10)